In [5]:
import os
import sys
os.chdir('/home/peitian_zhang/Codes/NR')
sys.path.append('/home/peitian_zhang/Codes/NR')

import math
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime
from torchtext.vocab import GloVe
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from utils.MIND import MIND_iter,MIND_map
from utils.utils import getLoss,getLabel,constructBasicDict,run_eval,run_train
from models.KNRM import KNRMModel

hparams = {
    'mode':'demo',
    'name':'knrm',
    'epochs':10,
    'batch_size':5,
    'title_size':20,
    'his_size':20,
    'npratio':4,
    'embedding_dim':300,
    'kernel_num':11,
    'metrics':'group_auc,ndcg@5,ndcg@10,mean_mrr',
    'device':'cuda:0',
    'attrs': ['title'],
}

news_file_train = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_train/news.tsv'
news_file_test = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_dev/news.tsv'
news_file_pair = (news_file_train,news_file_test)

behavior_file_train = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_train/behaviors.tsv'
behavior_file_test = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_dev/behaviors.tsv'
behavior_file_pair = (behavior_file_train,behavior_file_test)

save_path = 'models/model_params/{}_{}_{}'.format(hparams['name'],hparams['mode'],hparams['epochs']) +'.model'

if not os.path.exists('data/dictionaries/vocab_{}_{}.pkl'.format(hparams['mode'],'_'.join(hparams['attrs']))):
    constructBasicDict(news_file_pair,behavior_file_pair,hparams['mode'],hparams['attrs'])

device = torch.device(hparams['device']) if torch.cuda.is_available() else torch.device("cpu")

dataset_train = MIND_map(hparams=hparams,news_file=news_file_train,behaviors_file=behavior_file_train)

dataset_test = MIND_iter(hparams=hparams,news_file=news_file_test,behaviors_file=behavior_file_test, mode='test')

vocab = dataset_train.vocab
embedding = GloVe(dim=300,cache='.vector_cache')
vocab.load_vectors(embedding)

loader_train = DataLoader(dataset_train,batch_size=hparams['batch_size'],shuffle=True,pin_memory=True,num_workers=3,drop_last=True)
loader_test = DataLoader(dataset_test,batch_size=hparams['batch_size'],pin_memory=True,num_workers=0,drop_last=True)

In [6]:
knrmModel = KNRMModel(hparams=hparams,vocab=vocab).to(device)
knrmModel.train()

KNRMModel(
  (CosSim): CosineSimilarity()
  (softmax): Softmax(dim=-1)
  (learningToRank): Linear(in_features=400, out_features=1, bias=True)
)

In [10]:
a = torch.zeros((1,2,3),device=device)
a, knrmModel._kernel_pooling(a)

(tensor([[[0., 0., 0.],
          [0., 0., 0.]]], device='cuda:0'),
 tensor([[-46.0517, -46.0517, -46.0517, -33.8028, -22.8028, -13.8028,  -6.8028,
           -1.8028,   1.1972,   2.1972,   1.1972,  -1.8028,  -6.8028, -13.8028,
          -22.8028, -33.8028, -46.0517, -46.0517, -46.0517, -46.0517]],
        device='cuda:0'))

In [4]:
if knrmModel.training:
    print("training...")
    loss_func = getLoss(knrmModel)
    optimizer = optim.Adam(knrmModel.parameters(),lr=0.001)
    knrmModel = run_train(knrmModel,loader_train,optimizer,loss_func,epochs=hparams['epochs'], interval=10)

training...
epoch 0 , step 390 , loss: 3.4381: : 400it [01:20,  4.98it/s]
epoch 1 , step 390 , loss: 1.7071: : 400it [01:21,  4.91it/s]
epoch 2 , step 390 , loss: 1.6753: : 400it [01:24,  4.73it/s]
epoch 3 , step 390 , loss: 1.6454: : 400it [01:26,  4.64it/s]
epoch 4 , step 390 , loss: 1.6731: : 400it [01:26,  4.61it/s]
epoch 5 , step 390 , loss: 1.6560: : 400it [01:30,  4.44it/s]
epoch 6 , step 390 , loss: 1.6436: : 400it [01:27,  4.57it/s]
epoch 7 , step 390 , loss: 1.6694: : 400it [01:28,  4.50it/s]
epoch 8 , step 390 , loss: 1.6409: : 400it [01:28,  4.51it/s]
epoch 9 , step 390 , loss: 1.6431: : 400it [01:29,  4.49it/s]


In [8]:
print("evaluating...")
knrmModel.eval()
knrmModel.cdd_size = 1

run_eval(knrmModel,loader_test)

6it [00:00, 56.60it/s]evaluating...
3622it [00:41, 87.49it/s]
evaluation results:{'group_auc': 0.5005, 'ndcg@5': 0.2197, 'ndcg@10': 0.2934, 'mean_mrr': 0.2144}


{'group_auc': 0.5005, 'ndcg@5': 0.2197, 'ndcg@10': 0.2934, 'mean_mrr': 0.2144}